In [21]:
import pandas as pd
import matplotlib.pyplot as plt

In [22]:
recorded_driving_data = pd.read_csv('....')
recorded_driving_data['Timestamp'] = pd.to_datetime(recorded_driving_data['Timestamp'])

In [23]:
recorded_driving_data.info()

In [24]:
excellent_rhombus = {
    "positive_point_on_X_axis": (0.05,0),
    "negative_point_on_X_axis": (-0.05,0),
    "positive_point_on_Y_axis": (0,0.28),
    "negative_point_on_Y_axis": (0,-0.28)
}

acceptable_rhombus = {
    "positive_point_on_X_axis": (0.98,0),
    "negative_point_on_X_axis": (-0.98,0),
    "positive_point_on_Y_axis": (0,1.23),
    "negative_point_on_Y_axis": (0,-1.23)
}

so_and_so_rhombus = {
    "positive_point_on_X_axis": (1.87,0),
    "negative_point_on_X_axis": (-1.87,0),
    "positive_point_on_Y_axis": (0,2.12),
    "negative_point_on_Y_axis": (0,-2.12)
}

def compute_rhombus(rhombus):
    rhombus_half_diagonal1_length = abs(rhombus["positive_point_on_X_axis"][0])
    rhombus_half_diagonal2_length = abs(rhombus["positive_point_on_Y_axis"][1])
    rhombus['half_diagonal1_length'] = rhombus_half_diagonal1_length
    rhombus['half_diagonal2_length'] = rhombus_half_diagonal2_length
    return rhombus

excellent_rhombus = compute_rhombus(excellent_rhombus)
acceptable_rhombus = compute_rhombus(acceptable_rhombus)
so_and_so_rhombus = compute_rhombus(so_and_so_rhombus)

def point_inside_or_on_centered_rhombus(point_x, point_y, rhombus_half_diagonal1_length, rhombus_half_diagonal2_length):
    return abs(point_x / rhombus_half_diagonal1_length) + abs(point_y / rhombus_half_diagonal2_length) <= 1


def point_outside_centered_rhombus(point_x, point_y, rhombus_half_diagonal1_length, rhombus_half_diagonal2_length):
    return abs(point_x / rhombus_half_diagonal1_length) + abs(point_y / rhombus_half_diagonal2_length) > 1

def remove_points_inside_and_outside_rhombus_boundaries(inside_rhombus,outside_rhombus,label,dataframe,dataframe_column_name,dataframe_x_axis_column_name,dataframe_y_axis_column_name):
    for index,row in dataframe.iterrows():
        if row[dataframe_column_name] == label:
            point_x = row[dataframe_x_axis_column_name]
            point_y = row[dataframe_y_axis_column_name]
            if (point_inside_or_on_centered_rhombus(point_x,
                                                    point_y,
                                                    outside_rhombus['half_diagonal1_length'],
                                                    outside_rhombus['half_diagonal2_length'])
                    == True
                    and
                    point_outside_centered_rhombus(point_x,
                                                   point_y,
                                                   inside_rhombus['half_diagonal1_length'],
                                                   inside_rhombus['half_diagonal2_length'])
                    == True):
                pass
            else:
                dataframe = dataframe.drop(index)
    return dataframe

def classify_points_inside_rhombus_boundaries(rhombus,dataframe,dataframe_x_axis_column_name,dataframe_y_axis_column_name,label):
    for index,row in dataframe.iterrows():
        point_x = row[dataframe_x_axis_column_name]
        point_y = row[dataframe_y_axis_column_name]
        if (point_inside_or_on_centered_rhombus(point_x,
                                                point_y,
                                                rhombus['half_diagonal1_length'],
                                                rhombus['half_diagonal2_length'])
                == True):
            dataframe.at[index,'Label'] = label
    return dataframe

In [25]:
recorded_driving_data['Label'] = str('Uncomfortable')

recorded_driving_data = classify_points_inside_rhombus_boundaries(so_and_so_rhombus,recorded_driving_data,'Lateral acceleration','Longitudinal acceleration','So and So')

recorded_driving_data = classify_points_inside_rhombus_boundaries(acceptable_rhombus,recorded_driving_data,'Lateral acceleration','Longitudinal acceleration','Acceptable')

recorded_driving_data = classify_points_inside_rhombus_boundaries(excellent_rhombus,recorded_driving_data,'Lateral acceleration','Longitudinal acceleration','Excellent')

Plot of the labeled Data:

In [26]:
color_map = {
    'Excellent': 'green',
    'Acceptable': 'yellow',
    'So and So': 'orange',
    'Uncomfortable': 'red',
}

axis = recorded_driving_data.plot(kind='scatter', x='Lateral acceleration', y='Longitudinal acceleration')
colors = recorded_driving_data['Label'].map(color_map)
axis.scatter(recorded_driving_data['Lateral acceleration'], recorded_driving_data['Longitudinal acceleration'], color=colors)

max_lateral = recorded_driving_data['Lateral acceleration'].max()
max_longitudinal = recorded_driving_data['Longitudinal acceleration'].max()
maximum_axis_value = max_lateral if max_lateral > max_longitudinal else max_longitudinal

axis.set_xlim(-maximum_axis_value-0.5,maximum_axis_value+0.5)
axis.set_ylim(-maximum_axis_value-0.5,maximum_axis_value+0.5)

plt.grid(True)
plt.xlabel('Lateral Acceleration (m/s^2)')
plt.ylabel('Longitudinal Acceleration (m/s^2)')
plt.title('Baseline Labeled Acceleration Data')
plt.show()

In [27]:
recorded_driving_data.to_csv('../../Data/Verified/BaseLineLabeledWithoutSelfLabeledValues/RecordedDrivingData.csv',index=False)